<a href="https://colab.research.google.com/github/20pa1a1268/basicml/blob/main/Copy_of_stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,  classification_report

In [6]:
df = pd.read_csv('set2.csv', encoding='latin-1')
df["FID"]

0        853-779-238.5185467-17.50703998
1        857-858-62.66999758-14.47982622
2        790-783-157.8688262-21.40448992
3        564-555-45.09365651-8.273289523
4        778-775-52.30224246-10.91331722
                      ...               
49994    844-840-60.70597559-13.27512212
49995    733-738-78.10166165-10.44881842
49996     762-765-52.25012493-9.50995717
49997    717-717-33.59767885-9.332227094
49998     885-902-80.3400147-13.43424497
Name: FID, Length: 49999, dtype: object

In [7]:
df.fillna(value=1,inplace=True)
df.isnull().sum()

MEANRR       0
MEDIANRR     0
SDRR         0
RMSSD        0
condition    0
MEAN_RR      0
MEDIAN_RR    0
FID1         0
FID          0
dtype: int64

In [8]:
def apply_results(condition):
            if (condition == 'no stress'):
                return 0  # No Stress
            elif (condition == 'interruption' or condition=="time pressure"):
                return 1  # Stress
            else:
                return 0
df['results'] = df['condition'].apply(apply_results)
x = df["FID"]
y=df["results"]
cv = CountVectorizer(lowercase=False, strip_accents='unicode', ngram_range=(1, 1))
x = cv.fit_transform(df["FID"])

In [9]:
models = []
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)
X_train.shape, X_test.shape, y_train.shape

((39999, 101224), (10000, 101224), (39999,))

In [10]:
print("Naive Bayes")
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(X_train, y_train)
predict_nb = NB.predict(X_test)
naivebayes = accuracy_score(y_test, predict_nb) * 100
print(naivebayes)
print(confusion_matrix(y_test, predict_nb))
print(classification_report(y_test, predict_nb))
models.append(('naive_bayes', NB))


Naive Bayes
73.69
[[4224 1176]
 [1455 3145]]
              precision    recall  f1-score   support

           0       0.74      0.78      0.76      5400
           1       0.73      0.68      0.71      4600

    accuracy                           0.74     10000
   macro avg       0.74      0.73      0.73     10000
weighted avg       0.74      0.74      0.74     10000



In [11]:
print("SVM")
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train, y_train)
predict_svm = lin_clf.predict(X_test)
svm_acc = accuracy_score(y_test, predict_svm) * 100
print(svm_acc)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, predict_svm))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, predict_svm))
models.append(('svm', lin_clf))

SVM
76.69
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.77      0.82      0.79      5400
           1       0.77      0.71      0.74      4600

    accuracy                           0.77     10000
   macro avg       0.77      0.76      0.76     10000
weighted avg       0.77      0.77      0.77     10000

CONFUSION MATRIX
[[4414  986]
 [1345 3255]]


In [12]:
print("Logistic Regression")
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train, y_train)
y_pred = reg.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, y_pred) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, y_pred))
models.append(('logistic', reg))

Logistic Regression
ACCURACY
75.99000000000001
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.76      0.80      0.78      5400
           1       0.76      0.71      0.73      4600

    accuracy                           0.76     10000
   macro avg       0.76      0.76      0.76     10000
weighted avg       0.76      0.76      0.76     10000

CONFUSION MATRIX
[[4345 1055]
 [1346 3254]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
print("Decision Tree Classifier")
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtcpredict = dtc.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, dtcpredict) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, dtcpredict))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, dtcpredict))

Decision Tree Classifier
ACCURACY
90.05
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.90      0.91      0.91      5400
           1       0.90      0.89      0.89      4600

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

CONFUSION MATRIX
[[4934  466]
 [ 529 4071]]


In [14]:
print("Convolution Neural Network (CNN)")
from sklearn.neural_network import MLPClassifier
mlpc = MLPClassifier().fit(X_train, y_train)
y_pred = mlpc.predict(X_test)
print("ACCURACY")
print(accuracy_score(y_test, y_pred) * 100)
print("CLASSIFICATION REPORT")
print(classification_report(y_test, y_pred))
print("CONFUSION MATRIX")
print(confusion_matrix(y_test, y_pred))
models.append(('MLPClassifier', mlpc))




Convolution Neural Network (CNN)
ACCURACY
78.23
CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       0.78      0.83      0.80      5400
           1       0.78      0.73      0.76      4600

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000

CONFUSION MATRIX
[[4468  932]
 [1245 3355]]
